# NDNc Consumer 

## Configure Fabric Environment

In [1]:
import os

os.environ['FABRIC_PROJECT_ID']='<project-id>'
os.environ['FABRIC_BASTION_HOST'] = 'bastion-1.fabric-testbed.net'
os.environ['FABRIC_BASTION_USERNAME']='<user-name>'
os.environ['FABRIC_BASTION_KEY_LOCATION']=os.environ['HOME']+'/work/.ssh/fabric_bastion_key'
os.environ['FABRIC_SLICE_PRIVATE_KEY_FILE']=os.environ['HOME']+'/.ssh/id_rsa'
os.environ['FABRIC_SLICE_PUBLIC_KEY_FILE']=os.environ['HOME']+'/.ssh/id_rsa.pub'

### Import FABRIC API

In [2]:
import json
import traceback

from fabrictestbed_extensions.fablib.fablib import fablib

## Configure Slice Parameters



In [ ]:
import time

try:
    sliceNum = int(time.time())
    sliceName = 'ndnc'
    # FABRIC site to allocate slice; it must have IPv4 to reach the router
    SITE = 'TACC'
    # remote router on /ndn network, written as IPv4 address (not hostname) and UDP port
    ROUTER_IP, ROUTER_PORT = '128.196.203.36', 6363

    slice = fablib.new_slice(name=sliceName)
    nodeC = slice.add_node(name='C', site=SITE, cores=12,
                           ram=32, disk=80, image='default_ubuntu_22')
    nodeC.add_component(model='NIC_Basic', name='nic1')
    slice.submit()
    
    slice = fablib.get_slice(name=sliceName)
    nodeC = slice.get_node(name='C')
    print(nodeC.get_ssh_command())
    
except Exception as e:
    print(f"Slice Fail: {e}")

### Install Packages

In [ ]:
nodeC.execute(f'''
    echo "deb [arch=amd64 trusted=yes] https://nfd-nightly-apt.ndn.today/ubuntu jammy main" | sudo tee /etc/apt/sources.list.d/nfd-nightly.list
    sudo apt update
    sudo DEBIAN_FRONTEND=noninteractive apt full-upgrade -y
    sudo DEBIAN_FRONTEND=noninteractive apt install -y --no-install-recommends jq libibverbs-dev linux-image-generic ndnsec ndnpeek nfd apt-transport-https ca-certificates curl software-properties-common ndn-tools
    
    curl -fsSL https://download.docker.com/linux/ubuntu/gpg | sudo gpg --dearmor -o /usr/share/keyrings/docker-archive-keyring.gpg
    echo "deb [arch=amd64 signed-by=/usr/share/keyrings/docker-archive-keyring.gpg] https://download.docker.com/linux/ubuntu $(lsb_release -cs) stable" | sudo tee /etc/apt/sources.list.d/docker.list
    sudo apt update
    sudo DEBIAN_FRONTEND=noninteractive apt install -y --no-install-recommends docker-ce
    sudo loginctl enable-linger {nodeC.get_username()}
    
    sudo groupadd docker
    sudo usermod -aG docker $USER
    newgrp docker
    
    cat ./pw.txt | docker login --username sankalpatimilsina --password-stdin 
    
    wget https://raw.githubusercontent.com/DPDK/dpdk/main/usertools/dpdk-hugepages.py
    chmod +x dpdk-hugepages.py
    sudo mv dpdk-hugepages.py /usr/bin/
    
    sudo reboot
''')
slice.wait_ssh(progress=True)

nodeC.execute('''
    docker pull sankalpatimilsina/ndnc:nov-11
    docker pull sankalpatimilsina/ndn-dpdk-apr-27
    docker tag sankalpatimilsina/ndn-dpdk-apr-27 ndn-dpdk
''')

### Set NDN-DPDK Forwarder args

In [5]:
import shlex

UCLA_ROUTER_IP = '131.179.196.48'
UCLA_ROUTER_PORT = 6363

FW_ACTIVATE = {
    'mempool': {
        'DIRECT': {'capacity': 2**20-1, 'dataroom': 9200},
        'INDIRECT': {'capacity': 2**21-1},
    },
}
FW_UDP = {
    'scheme': 'udp',
    'remote': f'{UCLA_ROUTER_IP}:{UCLA_ROUTER_PORT}',
    'mtu': 1420,
}

### Run NDN-DPDK Forwarder and NDNc

In [ ]:
import shlex

stdout, stderr = nodeC.execute(f'''
    sudo dpdk-hugepages.py --clear
    sudo dpdk-hugepages.py --pagesize 1G --setup 30G
    
    echo "Launching forwarder container...."
    sudo docker ps -q --filter "name=fw" | grep -q . && sudo docker stop fw
    sudo docker ps -q -a --filter "name=fw" | grep -q . && sudo docker rm fw
    
    sudo docker run -d --name fw \
          --network host \
          --privileged \
          --mount type=bind,source=/dev/hugepages,target=/dev/hugepages \
          --mount type=volume,source=run-ndn,target=/run/ndn \
          ndn-dpdk

    sleep 5
    
    echo "Activating forwarder...."
    echo {shlex.quote(json.dumps(FW_ACTIVATE))} | sudo docker run -i --rm \
    --privileged \
    --network host \
    --mount type=bind,source=/dev/hugepages,target=/dev/hugepages \
    --mount type=volume,source=run-ndn,target=/run/ndn \
    ndn-dpdk ndndpdk-ctrl activate-forwarder
    
    echo "Creating face to next ndn-dpdk forwarder...."
    FW_UDP_FACE=$(echo {shlex.quote(json.dumps(FW_UDP))} | sudo docker run -i --rm \
    --privileged --network host \
    ndn-dpdk \
    ndndpdk-ctrl create-face)
    echo $FW_UDP_FACE
    
    echo "Inserting FIB entry for /fileserver to nexthop...."
    sudo docker run --privileged --network host \
    -i --rm ndn-dpdk \
    ndndpdk-ctrl insert-fib --name / --nexthop $(echo $FW_UDP_FACE | jq -r .id)
        
    #$CTRL_FW list-faces
    #$CTRL_FW list-fib
    
    sudo docker ps -q --filter "name=ndnc" | grep -q . && sudo docker stop ndnc
    sudo docker ps -q -a --filter "name=ndnc" | grep -q . && sudo docker rm ndnc
    sudo docker run -d --name ndnc --network host \
    --privileged \
    --mount type=bind,source=/dev/hugepages,target=/dev/hugepages \
    --mount type=volume,source=run-ndn,target=/run/ndn \
    sankalpatimilsina/ndnc:nov-11 tail -f /dev/null 
    ''')
print(f'{stdout}\n{stderr}')

## Delete Slice

In [14]:
try:
    slice = fablib.get_slice(name=sliceName)
    slice.delete()
except Exception as e:
    print(f"Fail: {e}")